In [1]:
from dotenv import load_dotenv
import os
import oracledb
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor, NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder, PolynomialFeatures
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.feature_selection import SelectKBest, f_regression
import json


In [34]:
load_dotenv() 

# Load configuration from env variables
try:
    user = os.environ.get("USER")
    password = os.environ.get("PASSWORD")
    db_dsn = os.environ.get("DB_DSN")
    wallet_location = os.environ.get("WALLET_LOCATION")
except:
    raise RuntimeError("Unable to find configuration in env variables")


try:
    # Use raw string for password to avoid escaping issues
    connection = oracledb.connect(
        user=user,
        password=password,
        dsn=db_dsn,
        config_dir=wallet_location,
        wallet_location=wallet_location,
        wallet_password=password
    )
    print("Connection successful!")
except oracledb.Error as error:
    print(f"Connection failed with error: {error}")

cursor = connection.cursor()

# Define the query
query = f"SELECT * FROM zip_codes"

cursor.execute(query)

rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
cursor.close()


Connection successful!


In [3]:
# Get unique combinations and first ZIP per county
county_state_zip_list = (df[['STATE', 'COUNTY', 'ZIPCODE']]
    .drop_duplicates()
    .groupby(['STATE', 'COUNTY'])
    .agg({'ZIPCODE': 'first'})
    .reset_index()
)

# Display result
print("Counties with State and ZIP:")
display(county_state_zip_list)

ziplist = county_state_zip_list['ZIPCODE'].tolist()
print(ziplist)


Counties with State and ZIP:


,STATE,COUNTY,ZIPCODE
0,Alabama,Autauga,36003
1,Alabama,Baldwin,36507
2,Alabama,Barbour,36016
3,Alabama,Bibb,35034
4,Alabama,Blount,35031
...,...,...,...
3129,Wyoming,Sweetwater,82901
3130,Wyoming,Teton,83001
3131,Wyoming,Uinta,82930
3132,Wyoming,Washakie,82401


['36003', '36507', '36016', '35034', '35031', '36029', '36030', '36201', '36854', '35959', '35045', '36904', '36436', '35082', '36262', '36323', '35616', '36401', '35072', '36028', '36009', '35019', '36311', '36701', '35961', '36020', '36426', '35901', '35542', '35571', '36313', '35443', '35441', '36310', '36301', '35740', '35005', '35544', '35610', '355XX', '36801', '35611', '36032', '36031', '35741', '36732', '35543', '35016', '36505', '36425', '36013', '35601', '36756', '35442', '36005', '36263', '36856', '35007', '35004', '35459', '35014', '35010', '35111', '35063', '36513', '36435', '35540', '99553', '99546', '99501', '99551', '99613', '99704', '99555', '99701', '99827', '99801', '99556', '99901', '99550', '99548', '99645', '99659', '99721', '99727', '99903', '99835', '99820', '99732', '99566', '99554', '99830', '99689', '99558', '85920', '85602', '85931', '85501', '85530', '85533', '85325', '85268', '85360', '85901', '85321', '85272', '85611', '85324', '85333', '72003', '71635', 

In [ ]:
#Runs in about 83 minutes

from tqdm import tqdm
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import random

# Setup single browser
options = Options()
options.add_argument('--headless')
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

state_groups = county_state_zip_list.groupby('STATE')


try:
    # Navigate once
    driver.get("https://smartasset.com/taxes/property-taxes")
    
    # Initialize master DataFrame
    master_tax_data = pd.DataFrame(columns=['state', 'zipcode', 'county', 'tax_rate'])
    
    # Process all states
    for state, state_data in tqdm(state_groups, desc="Processing States"):
        # Process zipcodes for this state
        for zipcode in tqdm(state_data['ZIPCODE'].tolist(), desc=f"Processing {state}"):
            try:
                # Random delay between requests
                time.sleep(random.uniform(1.0, 2.0))
                
                # Input zipcode
                input_element = wait.until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='ud-current-location-display']"))
                )
                input_element.clear()
                input_element.send_keys(str(zipcode))
                input_element.send_keys(Keys.RETURN)
                
                # Extract data with retry
                for _ in range(3):  # 3 retries
                    try:
                        tax_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "tax-rate")))
                        tax_rate = float(tax_element.find_element(By.CLASS_NAME, "value").text.rstrip('%'))
                        county = tax_element.find_element(By.CLASS_NAME, "county").text.strip('()')
                        break
                    except Exception:
                        time.sleep(1)
                        continue
                
                # Add to DataFrame
                master_tax_data.loc[len(master_tax_data)] = {
                    'state': state,
                    'zipcode': zipcode,
                    'county': county,
                    'tax_rate': tax_rate
                }
                
                # Save every 100 records
                if len(master_tax_data) % 100 == 0:
                    master_tax_data.to_csv(f'tax_rates_checkpoint.csv', index=False)
                
            except Exception as e:
                print(f"Error processing zipcode {zipcode}: {e}")
                continue
                
finally:
    driver.quit()
    master_tax_data.to_csv('tax_rates_final.csv', index=False)

Processing States: 100%|██████████| 51/51 [1:23:06<00:00, 97.77s/it] 


In [19]:
load_dotenv() 

# Load configuration from env variables
try:
    user = os.environ.get("USER")
    password = os.environ.get("PASSWORD")
    db_dsn = os.environ.get("DB_DSN")
    wallet_location = os.environ.get("WALLET_LOCATION")
except:
    raise RuntimeError("Unable to find configuration in env variables")


try:
    # Use raw string for password to avoid escaping issues
    connection = oracledb.connect(
        user=user,
        password=password,
        dsn=db_dsn,
        config_dir=wallet_location,
        wallet_location=wallet_location,
        wallet_password=password
    )
    print("Connection successful!")
except oracledb.Error as error:
    print(f"Connection failed with error: {error}")

cursor = connection.cursor()

# Define the query
query = f"SELECT * FROM zip_codes"

cursor.execute(query)

rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
cursor.close()


Connection successful!


In [20]:
df_copy = df.copy()


In [42]:
# 1. Load and prepare tax rates
tax_rates = pd.read_csv("tax_rates_final.csv")

zip_county_mapping = tax_rates[['zipcode']].drop_duplicates()

df_lkup = df_copy.merge(
    zip_county_mapping,
    left_on='ZIPCODE',
    right_on='zipcode',
    how='inner'
)

df_lkup["cnty_zip"] = df_lkup["ZIPCODE"]
df_lkup = df_lkup[["STATE", "COUNTY", "cnty_zip"]]

zip_db = df_copy.merge(
    df_lkup,
    left_on=["STATE", "COUNTY"],
    right_on=["STATE", "COUNTY",],
    how='inner'
)

final = zip_db.merge(
    tax_rates[['zipcode', 'tax_rate']],
    left_on='cnty_zip',
    right_on='zipcode',
    how='left'
)

final = final.drop(columns=['cnty_zip', 'zipcode'])
display(final[final['ZIPCODE'] == "98177"])

,STATE_FIPS,STATE,STATE_ABBR,ZIPCODE,COUNTY,CITY,tax_rate
25380,53,Washington,WA,98177,King,Seattle,0.761


In [43]:
# Create comprehensive ZIP mapping
zip_info_dict = {}

for _, row in final.iterrows():
    zip_code = str(row['ZIPCODE'])
    zip_info_dict[zip_code] = {
        'state': {
            'fips': row['STATE_FIPS'],
            'name': row['STATE'],
            'abbr': row['STATE_ABBR']
        },
        'location': {
            'county': row['COUNTY'],
            'city': row['CITY']
        },
        'tax_rate': float(row['tax_rate']) if pd.notna(row['tax_rate']) else None
    }

# Export formatted JSON
with open('zip_tax_info.json', 'w') as f:
    json.dump(zip_info_dict, f, indent=2)

# Verify structure
print(f"Total ZIP codes mapped: {len(zip_info_dict)}")
print("\nSample structure:")
sample = dict(list(zip_info_dict.items())[0:1])
print(json.dumps(sample, indent=2))


Total ZIP codes mapped: 33048

Sample structure:
{
  "41822": {
    "state": {
      "fips": 21,
      "name": "Kentucky",
      "abbr": "KY"
    },
    "location": {
      "county": "Knott",
      "city": "Hindman"
    },
    "tax_rate": 1.044
  }
}
